<style>
    .title {
        font-weight: 'bold';
    }
</style>

# <span class="title">08.1 - Création du dataset de données traitées pour entrainement </br> avec criticités équilibrées</span>
(sans encodage, ni normalisation)

Ce notebook génère 1 csv :

- dataset_for_training.csv : analyse et nettoyage des variables explicatives

Etapes : 




## A) Imports

In [16]:
import os
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
# from ydata_profiling import ProfileReport # pip install ydata_profiling ipywidgets
import matplotlib.pyplot as plt

In [17]:
# source path to datasets
path = '../data/'
data = 'dataset_for_preprocess_criticality_balanced_07.1.csv'
save_csv = '../data/dataset_for_training_criticality_balanced_08.1.csv'

In [18]:
# création d'un dataframe à partir du csv de données
df = pd.read_csv(os.path.join(path, data), header=0, parse_dates=True, index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 536 entries, 0 to 777
Data columns (total 40 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   started_at                                            536 non-null    object 
 1   ended_at                                              536 non-null    object 
 2   paperHeight_job                                       536 non-null    int64  
 3   paperWidth_job                                        536 non-null    int64  
 4   scanner_mode                                          536 non-null    int64  
 5   bars_job                                              536 non-null    int64  
 6   copies_per_run                                        536 non-null    int64  
 7   jobId                                                 536 non-null    int64  
 8   total_copies_requested                                536 no

In [19]:
df.columns

Index(['started_at', 'ended_at', 'paperHeight_job', 'paperWidth_job',
       'scanner_mode', 'bars_job', 'copies_per_run', 'jobId',
       'total_copies_requested', 'LED', 'bars_event', 'drops', 'dithering',
       'deadPixelsOffset', 'operator_level', 'operator', 'speed_ifoil',
       'ifoil', 'optifoil_ifoil', 'stampAreas_ifoil', 'heater1Enabled_ifoil',
       'heater1Temperature_ifoil', 'x_imageLayout', 'y_imageLayout',
       'paperWidth_event', 'paperHeight_event', 'speed', 'power_irDryers',
       'power_uvDryers', 'leftMargin_remoteScannerRegistration',
       'blueScore_fullScannerMode_remoteScannerRegistration',
       'greenScore_fullScannerMode_remoteScannerRegistration',
       'mode_remoteScannerRegistration', 'jobState', 'total_copies',
       'varnishConsumptionVarnish_3d', 'index', 'timestamp',
       'criticality_events', 'identification_events'],
      dtype='object')

## B) Analyse des variables

In [20]:
# remise à zero des index
df.reset_index(drop=True,inplace=True)

In [21]:
# suppression des colonnes de metrics
# metrics_cols = [
#     'status', 
#     'source_events', 
#     'timestamp', 
#     'criticality_events', 
#     'name_modules', 
#     'type_modules',
#     'generation_modules', 
#     'value_counters_modules', 
#     'name_counters_modules',
#     'name_connected_operators',
#     'level_connected_operators',
#     'varnishLevelsTargetvolume',
#     'varnishLevelsTotalvolume'
#     ]
metrics_cols = [
    'timestamp', 
    'identification_events', 
    'index',
    ]
df.drop(columns=metrics_cols, axis=0, inplace=True)

In [22]:
# suppression des colonnes en doublon entre job_events et jobs
job_events_cols = [
    'bars_event',
    'paperWidth_event', 
    'paperHeight_event'
    ]
df.drop(columns=job_events_cols, axis=0, inplace=True)

In [23]:
# suppression des colonnes de variables catégorielles
cat_cols = [
    'operator_level', 
    'operator',
    'jobState',
    'jobId'
   ]
df.drop(columns=cat_cols, axis=0, inplace=True)

In [24]:
# suppression des colonnes datetime et ajout d'une colonne duration
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])
# Calculer la différence de temps entre les deux colonnes
df['duration'] = (df['ended_at'] - df['started_at']).dt.total_seconds()
df.drop(columns=['started_at', 'ended_at'], axis=0, inplace=True)

In [25]:
df.head(3)

,paperHeight_job,paperWidth_job,scanner_mode,bars_job,copies_per_run,total_copies_requested,LED,drops,dithering,deadPixelsOffset,speed_ifoil,ifoil,optifoil_ifoil,stampAreas_ifoil,heater1Enabled_ifoil,heater1Temperature_ifoil,x_imageLayout,y_imageLayout,speed,power_irDryers,power_uvDryers,leftMargin_remoteScannerRegistration,blueScore_fullScannerMode_remoteScannerRegistration,greenScore_fullScannerMode_remoteScannerRegistration,mode_remoteScannerRegistration,total_copies,varnishConsumptionVarnish_3d,criticality_events,duration
0,520,740,1,2,0,350,10,1,False,0,36.0,False,True,"[{'id': 1, 'end': 519, 'start': 0, 'height': 5...",False,115,70,0,500,45,100,0,16,16,1,0,0.0,ERROR,74.200
1,520,740,1,2,0,350,10,1,False,0,36.0,False,True,"[{'id': 1, 'end': 519, 'start': 0, 'height': 5...",False,115,70,0,500,45,100,0,16,16,1,0,0.0,ERROR,74.200
2,520,740,1,2,0,350,10,1,False,0,36.0,False,True,"[{'id': 1, 'end': 519, 'start': 0, 'height': 5...",False,115,70,0,500,45,100,0,16,16,1,0,0.0,INFO,308.975


### a) Profiling report

In [26]:
# profile = ProfileReport(df, title="Profiling Report")
# profile.to_notebook_iframe()

## C) Nettoyage

In [27]:
# supression des colonnes dont les valeurs sont constantes
contstant_cols = ['dithering']
df.drop(columns=contstant_cols, axis=0, inplace=True)

In [28]:
df.head(5)

,paperHeight_job,paperWidth_job,scanner_mode,bars_job,copies_per_run,total_copies_requested,LED,drops,deadPixelsOffset,speed_ifoil,ifoil,optifoil_ifoil,stampAreas_ifoil,heater1Enabled_ifoil,heater1Temperature_ifoil,x_imageLayout,y_imageLayout,speed,power_irDryers,power_uvDryers,leftMargin_remoteScannerRegistration,blueScore_fullScannerMode_remoteScannerRegistration,greenScore_fullScannerMode_remoteScannerRegistration,mode_remoteScannerRegistration,total_copies,varnishConsumptionVarnish_3d,criticality_events,duration
0,520,740,1,2,0,350,10,1,0,36.0,False,True,"[{'id': 1, 'end': 519, 'start': 0, 'height': 5...",False,115,70,0,500,45,100,0,16,16,1,0,0.0,ERROR,74.200
1,520,740,1,2,0,350,10,1,0,36.0,False,True,"[{'id': 1, 'end': 519, 'start': 0, 'height': 5...",False,115,70,0,500,45,100,0,16,16,1,0,0.0,ERROR,74.200
2,520,740,1,2,0,350,10,1,0,36.0,False,True,"[{'id': 1, 'end': 519, 'start': 0, 'height': 5...",False,115,70,0,500,45,100,0,16,16,1,0,0.0,INFO,308.975
3,520,740,1,2,0,350,10,1,0,36.0,False,True,"[{'id': 1, 'end': 519, 'start': 0, 'height': 5...",False,115,70,0,500,45,100,0,16,16,1,0,0.0,INFO,308.975
4,520,740,1,2,0,350,10,1,0,36.0,False,True,"[{'id': 1, 'end': 519, 'start': 0, 'height': 5...",False,115,70,0,500,45,100,0,16,16,1,0,0.0,INFO,197.449


<style>
    .important {color: #A5EF6E;}
    .purple { color: #EF6EE9;}
</style>

Données disponibles au démarrage d'un job :
- <span class='important'>1  thumbnail* : chemin vignette projet</span>
- 2  total_copies : nombre de copies imprimées
- 3  started_at : date de début
- 4  ended_at : date de fin
- <span class='important'>5  machineId* : identifiant machine</span>
- 6  speed : vitesse d'impression
- 7  operator : opérateur
- 8  operator_level : niveau de l'opérateur
- 9  first_page_image_path_on_machine* : chemin de l'image
- 10 paperHeight : hauteur du substrat
- 11 paperWidth : largeur du substrat
- 12 paperName : appelation du substrat
- <span class='important'>13 paperThickness* : épaisseur du substrat (valeur unique à 0)</span>
- 14 id_on_machine : identifiant unique du travail d'impression
- 15 total_copies_requested : nombre de copies demandées
- <span class='important'>16 job_thumbnail_id* : identifiant de vignette image</span>
- 17 uses_ifoil : impression utilisant de la dorure
- <span class='important'>18 uses_iper* : impression nécéssitant l'iper (valeur unique à True)</span>
- 19 scanner_mode : niveau de config du scanner
- 20 iper_bvar_count : compteur <span class='purple'># TODO vérifier la fusion notebook 06</span>
- 21 varnishConsumptionVarnish_3d : consommation de vernis en 3d
- <span class='important'>22 varnishConsumptionVarnish_2d* : consommation de vernis en 2d</span>

<span class='important'> *\*variables déjà supprimées dans le notebook 05 ou 06*</span>

In [29]:
columns_to_add = ['redScore_gridMode_remoteScannerRegistration',
                  'redScore_cropmarksMode_remoteScannerRegistration',
                  'redScore_fullScannerMode_remoteScannerRegistration']

df[columns_to_add] = 2561

In [31]:
# colonnes à conserver dispos dans jobs
jobs_cols_to_keep = [
    'duration',
    'total_copies',
    'speed',
    'paperHeight_job', 
    'paperWidth_job', 
    'total_copies_requested',
    'ifoil',
    'scanner_mode', 
    'bars_job', # TODO vérifier les valeurs de la variable d'origine iper_bvar_count
    'varnishConsumptionVarnish_3d'
    ]
# colonnes à conserver dispos dans job events avec le tag start
jobevents_cols_to_keep = [
    'LED',
    'drops', 
    'speed_ifoil',
    'optifoil_ifoil', 
    'stampAreas_ifoil',
    'heater1Enabled_ifoil', 
    'heater1Temperature_ifoil',
    'x_imageLayout',
    'y_imageLayout', 
    'power_irDryers', 
    'power_uvDryers',
    'leftMargin_remoteScannerRegistration',
    'redScore_gridMode_remoteScannerRegistration',
    'redScore_cropmarksMode_remoteScannerRegistration',
    'redScore_fullScannerMode_remoteScannerRegistration',
    'blueScore_fullScannerMode_remoteScannerRegistration',
    'greenScore_fullScannerMode_remoteScannerRegistration',
    'mode_remoteScannerRegistration'
    ]
# colonnes à conserver pour la prédiction
metrics_cols_to_keep = ['criticality_events']

# fusion des listes de colonnes à conserver
cols_to_keep = jobs_cols_to_keep + jobevents_cols_to_keep + metrics_cols_to_keep

# dataframe des variables conservées
df_to_encode = df[cols_to_keep].copy()

In [32]:
df_to_encode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536 entries, 0 to 535
Data columns (total 29 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   duration                                              536 non-null    float64
 1   total_copies                                          536 non-null    int64  
 2   speed                                                 536 non-null    int64  
 3   paperHeight_job                                       536 non-null    int64  
 4   paperWidth_job                                        536 non-null    int64  
 5   total_copies_requested                                536 non-null    int64  
 6   ifoil                                                 536 non-null    bool   
 7   scanner_mode                                          536 non-null    int64  
 8   bars_job                                              536 no

In [33]:
df_to_encode.drop(columns=['stampAreas_ifoil', 'total_copies', 'mode_remoteScannerRegistration'], axis=0, inplace=True)

In [34]:
df_to_encode.criticality_events.unique()

array(['ERROR', 'INFO', 'WARNING', 'NO_EVENT'], dtype=object)

## D) Output

In [35]:
df_to_encode.to_csv(save_csv)